<img src="ost_logo.png" width="240" height="240" align="right"/>
<div style="text-align: left"> <b> Machine Learning </b> <br> MSE FTP MachLe <br> 
<a href="mailto:christoph.wuersch@ost.ch"> Christoph Würsch </a> </div>

# The Naive Bayes Classifier
MSE FTP_MachLe, Christoph Würsch



Naive Bayes is a conditional probability model: given a problem instance to be classified, represented by a vector ${\displaystyle \mathbf {x} =(x_{1},\ldots ,x_{n})}$ representing some $n$ features (independent variables), it assigns to this instance probabilities

$${\displaystyle p(C_{k}\mid x_{1},\ldots ,x_{n})\,}$$

for each of $K$ possible outcomes or classes ${\displaystyle C_{k}}$ 

The problem with the above formulation is that if the number of features $n$ is large or if a feature can take on a large number of values, then basing such a model on probability tables is infeasible. We therefore reformulate the model to make it more tractable. Using __Bayes' theorem__, the conditional probability can be decomposed as:

$$ {\displaystyle p(C_{k}\mid \mathbf {x} )={\frac {p(C_{k})\ p(\mathbf {x} \mid C_{k})}{p(\mathbf {x} )}}\,} $$

Using Bayesian probability terminology, the above equation can be written as

$${\displaystyle {\text{posterior}}={\frac {{\text{prior}}\cdot {\text{likelihood}}}{\text{evidence}}}\,}$$

In practice, there is interest only in the numerator of that fraction, because the denominator does not depend on $C$ and the values of the features ${\displaystyle x_{i}}$ are given, so that the denominator is effectively constant. The numerator is equivalent to the joint probability model

${\displaystyle p(C_{k},x_{1},\ldots ,x_{n})\,}$ which can be rewritten as follows, using the chain rule for repeated applications of the definition of conditional probability:

$${\displaystyle {\begin{aligned}p(C_{k},x_{1},\ldots ,x_{n})&=p(x_{1},\ldots ,x_{n},C_{k})\\&=p(x_{1}\mid x_{2},\ldots ,x_{n},C_{k})\ p(x_{2},\ldots ,x_{n},C_{k})\\&=p(x_{1}\mid x_{2},\ldots ,x_{n},C_{k})\ p(x_{2}\mid x_{3},\ldots ,x_{n},C_{k})\ p(x_{3},\ldots ,x_{n},C_{k})\\&=\cdots \\&=p(x_{1}\mid x_{2},\ldots ,x_{n},C_{k})\ p(x_{2}\mid x_{3},\ldots ,x_{n},C_{k})\cdots p(x_{n-1}\mid x_{n},C_{k})\ p(x_{n}\mid C_{k})\ p(C_{k})\\\end{aligned}}}$$


__Now the "naive" conditional independence assumptions come into play: assume that all features in ${\displaystyle \mathbf {x} }$ are mutually independent, conditional on the category ${\displaystyle C_{k}}$.__ Under this assumption,

$${\displaystyle p(x_{i}\mid x_{i+1},\ldots ,x_{n},C_{k})=p(x_{i}\mid C_{k})\,}$$
Thus, the joint model can be expressed as

$${\displaystyle {\begin{aligned}p(C_{k}\mid x_{1},\ldots ,x_{n})&\varpropto p(C_{k},x_{1},\ldots ,x_{n})\\&=p(C_{k})\ p(x_{1}\mid C_{k})\ p(x_{2}\mid C_{k})\ p(x_{3}\mid C_{k})\ \cdots \\&=p(C_{k})\prod _{i=1}^{n}p(x_{i}\mid C_{k})\,,\end{aligned}}}$$

where ${\displaystyle \varpropto }$ denotes proportionality.

This means that __under the above independence assumptions__, the conditional distribution over the class variable $C$ is:

$${\displaystyle p(C_{k}\mid x_{1},\ldots ,x_{n})={\frac {1}{Z}}p(C_{k})\prod _{i=1}^{n}p(x_{i}\mid C_{k})}$$

where the evidence 
$${\displaystyle Z=p(\mathbf {x} )=\sum _{k}p(C_{k})\ p(\mathbf {x} \mid C_{k})}$$
is a scaling factor dependent only on ${\displaystyle x_{1},\ldots ,x_{n}}$ that is, a constant if the values of the feature variables are known.

Source: https://en.wikipedia.org/wiki/Naive_Bayes_classifier

## An introductory example: Mrs Marple plays Golf

Let's analyze the probabilty for Mrs Marple to play Golf on a given day. Mrs Marple has recorded her decisions on 14 different days. Depending on the `Temperature`, the `Outlook`, the `Humidity` and `Wind Condition` of the day, we want to predict, whether she is going to play Golf or not.

- $X_1$: `Temperature`
- $X_2$: `Outlook`
- $X_3$: `Humidity`
- $X_4$: `Windy?`


- $ \mathrm{dom}(X_1) = \left \lbrace \text{hot, cold, mild} \right \rbrace$
- $ \mathrm{dom}(X_2) = \left \lbrace \text{sunny, overcast, rain} \right \rbrace$
- $ \mathrm{dom}(X_3) = \left \lbrace \text{normal, high} \right \rbrace$
- $ \mathrm{dom}(X_4) = \left \lbrace \text{True, False} \right \rbrace$


The response $y$ is: `Play Golf?`.


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df=pd.read_excel('PlayGolf_NaiveBayes.xls')
df.head(20)

,Temperature,Outlook,Humidity,Windy,Play Golf?
0,hot,sunny,high,False,no
1,hot,sunny,high,True,no
2,hot,overcast,high,False,yes
3,cool,rain,normal,False,yes
4,cool,overcast,normal,True,yes
5,mild,sunny,high,False,no
6,cool,sunny,normal,False,yes
7,mild,rain,normal,False,yes
8,mild,sunny,normal,True,yes
9,mild,overcast,high,True,yes


### 1. Calculation of the Prior

The prior probabilty is the probability to play Golf at all. For this we only have to look at the last column.

$$p(y=\text{yes})=\frac{9}{14}$$
$$p(y=\text{no})=\frac{5}{14}$$


### 2. Calculation of the conditional probabilities $p(X_i \vert y)$

As a next step, we have to calculate all conditional probabilities, i.e. the probability for every feature $X_i$ given $y=\text{yes}$ and $y=\text{no}$. We can use a contingency table (`pandas.cosstab`) to calculate these probabilities.


In [5]:
pd.crosstab(df['Temperature'],df['Play Golf?'],margins=True,normalize=False)

Play Golf?,no,yes,All
Temperature,,,
cool,1,3,4
hot,2,2,4
mild,2,4,6
All,5,9,14


From this table, we can calculate the following:

- $p(X_1=\text{cool} \vert y=\text{yes} ) = \frac{3}{9}$
- $p(X_1=\text{hot } \vert y=\text{yes} )  = \frac{2}{9}$
- $p(X_1=\text{mild} \vert y=\text{yes} ) = \frac{4}{9}$

- $p(X_1=\text{cool} \vert y=\text{no } ) = \frac{1}{5}$
- $p(X_1=\text{hot } \vert y=\text{no } )  = \frac{2}{5}$
- $p(X_1=\text{mild} \vert y=\text{no } ) = \frac{2}{5}$



In [6]:
pd.crosstab(df['Outlook'],df['Play Golf?'],margins=True)

Play Golf?,no,yes,All
Outlook,,,
overcast,0,4,4
rain,2,3,5
sunny,3,2,5
All,5,9,14


From this table, we can calculate the following conditionals:

- $p(X_2=\text{overcast} \vert y=\text{yes} ) = \frac{4}{9}$
- $p(X_2=\text{rain } \vert y=\text{yes} )  = \frac{3}{9}$
- $p(X_2=\text{sunny} \vert y=\text{yes} ) = \frac{2}{9}$

- $p(X_2=\text{overcast} \vert y=\text{no } ) = \frac{0}{5}$
- $p(X_2=\text{rain } \vert y=\text{no } )  = \frac{2}{5}$
- $p(X_2=\text{sunny} \vert y=\text{no } ) = \frac{3}{5}$

In [7]:
pd.crosstab(df['Humidity'],df['Play Golf?'],margins=True)


Play Golf?,no,yes,All
Humidity,,,
high,4,3,7
normal,1,6,7
All,5,9,14


From this table, we can calculate the following conditionals:

- $p(X_3=\text{high  } \vert y=\text{yes} ) = \frac{3}{9}$
- $p(X_3=\text{normal} \vert y=\text{yes} )  = \frac{6}{9}$

- $p(X_3=\text{high  } \vert y=\text{no } ) = \frac{4}{5}$
- $p(X_3=\text{normal} \vert y=\text{no } )  = \frac{1}{5}$


In [8]:
pd.crosstab(df['Windy'],df['Play Golf?'],margins=True)

Play Golf?,no,yes,All
Windy,,,
False,2,6,8
True,3,3,6
All,5,9,14


And finally for the last feature $X_4$, we get

- $p(X_4=\text{False  } \vert y=\text{yes} ) = \frac{6}{9}$
- $p(X_4=\text{True} \vert y=\text{yes} )  = \frac{3}{9}$

- $p(X_4=\text{False  } \vert y=\text{no } ) = \frac{2}{5}$
- $p(X_4=\text{True} \vert y=\text{no } )  = \frac{3}{5}$

### 3. Making predictions

What is now the decision to play Golf under the the following conditions?

$$X= \left\lbrace X_1, X_2, X_3, X_4 \right \rbrace = \left \lbrace \text{hot, sunny, high, True} \right \rbrace$$

It is not necessary, to calculate the evidence $Z=p(X)$, because we can compare the following two nominators:

$$ p(y=\text{yes} \vert X ) \propto p(X_1 \vert y) \cdot p(X_2 \vert y) \cdot p(X_3 \vert y) \cdotp(X_4 \vert y) \cdot p(y)$$
and
$$ p(y=\text{no} \vert X ) \propto p(X_1 \vert \bar{y}) \cdot p(X_2 \vert \bar{y}) \cdot p(X_3 \vert \bar{y}) \cdotp(X_4 \vert y) \cdot p(\bar{y})$$


By looking at our tables, we get for the probability to play Golf $y=\text{yes}$:

$$ p(y=\text{yes} \vert X) = \frac{1}{Z} \cdot p(X_1 \vert y) \cdot p(X_2 \vert y) \cdot p(X_3 \vert y) \cdotp(X_4 \vert y) \cdot p(y) =
\frac{2}{9} \cdot \frac{2}{9} \cdot \frac{3}{9} \cdot \frac{3}{9} \cdot \frac{9}{14} = \frac{2^2}{9^2}\cdot \frac{1}{14 Z}$$

By looking at our tables, we get for the probability __not__ to play Golf: $y=\text{no}$:

$$ p(y=\text{not} \vert X) = \frac{1}{Z} \cdot p(X_1 \vert \bar{y}) \cdot p(X_2 \vert \bar{y}) \cdot p(X_3 \vert \bar{y}) \cdotp(X_4 \vert \bar{y}) \cdot p(\bar{y}) =
\frac{2}{5} \cdot \frac{2}{5} \cdot \frac{4}{5} \cdot \frac{3}{5} \cdot \frac{5}{14} = \frac{2^4}{5^3}\cdot \frac{1}{14 Z}$$

Now, we can compare these two probabilites and calculate the *ratio*:

$$\frac{p(y=\text{yes} \vert X = \left \lbrace \text{hot, sunny, high, True} \right \rbrace)}{p(y=\text{no} \vert X = \left \lbrace \text{hot, sunny, high, True} \right \rbrace)} = \frac{2^2 \cdot 5^3}{9^2 \cdot 2^4} = \frac{125}{4\cdot 81} =\frac{125}{324} < 1$$

### 4. What happens, if one of the counts is zero? 

In this case, the posterior $p(y \vert X)$ is zero as well. To avoid this, __Additive Smoothing (Laplace Smoothing)__ is normally applied to the data. 

$$p_i = \frac{n_i +1 }{n+k}$$

where:
- $n_i$: is the actual count of characteristic $i$ of the feature $X$, $(i=1 \dots k)$ conditioned on $y$
- $k$: is the number of classes (characteristics) in the considered feature $X$ conditioned on $y$
- $n$: is the total number of counts of this feature conditioned on $y$

Interested readers can have a look at: https://en.wikipedia.org/wiki/Additive_smoothing
